In [2]:
import time
import cv2
import os
import sys
import torch
import copy
import queue
import glob
import math
import random
import itertools
import threading
import numpy as np
import torch.nn as nn
from torch.backends import cudnn
from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess,sirxiapreprocess
from scipy.spatial.distance import pdist
from scipy.optimize import linear_sum_assignment

In [3]:
class KalmanFilter(object):
    def __init__(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y=0,0
        self.last_measurement = self.current_measurement = np.array((2,1),np.float32)
        np_bbox=np.asarray([x,y]).astype(np.float32)
        self.last_predicition = self.current_prediction = np_bbox.resize((2,1))#np.zeros((2,1),np.float32)
        self.kalman = cv2.KalmanFilter(4,2)
        #设置测量矩阵
        self.kalman.measurementMatrix = np.array([[1,0,0,0],[0,1,0,0]],np.float32)
        #设置转移矩阵
        self.kalman.transitionMatrix = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]],np.float32)
        #设置过程噪声协方差矩阵
        self.kalman.processNoiseCov = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],np.float32)*0.03 
    def move(self,x,y):
        #初始化
        self.last_measurement = self.current_measurement
        self.last_prediction = self.current_prediction
        #传递当前测量坐标值
        self.current_measurement = np.array([[np.float32(x)],[np.float32(y)]])
        #用来修正卡尔曼滤波的预测结果
        self.kalman.correct(self.current_measurement)
        # 调用kalman这个类的predict方法得到状态的预测值矩阵，用来估算目标位置
        current_prediction = self.kalman.predict()
        #上一次测量值
        lmx,lmy = self.last_measurement[0],self.last_measurement[1]
        #当前测量值
        cmx,cmy = self.current_measurement[0],self.current_measurement[1]
        #上一次预测值
        #lpx,lpy = last_prediction[0],last_prediction[1]
        #当前预测值
        cpx,cpy = current_prediction[0],current_prediction[1]
        return cpx,cpy
    def update(self,bbox):
        (x,y,w,h)=bbox
        self.x,self.y = self.move(x,y)
    def get(self):
        return self.x,self.y
        

In [94]:
class efficientdet:
    def __init__(self,compound_coef,force_input_size = 1920,huamanNum=1,dir_path='./walkingworkspace/train_all',NUM_EIGEN_FACES= 128,test=False):
        self.humanNum=huamanNum
        cudnn.fastest = True
        cudnn.benchmark = True
        self.bbox_save=[]
        self.id_save=[]
        self.test=test
        self.bbox_threshold=0.01#%bbox的阈值!<
        self.probability_threshold=0.3#%人体识别概率的阈值!<
        self.use_cuda = True 
        self.use_float16 = False
        self.threshold = 0.2
        self.iou_threshold = 0.2
        self.obj_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', '', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
            'cow', 'elephant', 'bear', 'zebra', 'giraffe', '', 'backpack', 'umbrella', '', '', 'handbag', 'tie',
            'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
            'skateboard', 'surfboard', 'tennis racket', 'bottle', '', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
            'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
            'cake', 'chair', 'couch', 'potted plant', 'bed', '', 'dining table', '', '', 'toilet', '', 'tv',
            'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
            'refrigerator', '', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
            'toothbrush']
        #self.device = torch.device('cuda')
        self.model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(self.obj_list))
        self.model.load_state_dict(torch.load(f'weights/efficientdet-d{compound_coef}.pth'))
        self.model.requires_grad_(False)
        self.model.eval()
        #self.model=nn.DataParallel(self.model)
        self.model.cuda()
        input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
        self.input_size =  force_input_size
        self.t=time.time()
        self.pca=PCA(dir_path,NUM_EIGEN_FACES)
        if self.test==True:
            self.pca.test('./walkingworkspace/test2020')
            self.label_test=[]
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        name = time.strftime('%Y.%m.%d',time.localtime(time.time()))
        self.out = cv2.VideoWriter('./savevideo/'+name+'.avi',fourcc, 30.0, (640,480))
        self.save={}
        self.frame=0
    def init_video(self):
        self.out.release()
        self.pca.clear_id()
        self.pca.frame=0
        name = time.strftime('%Y.%m.%d',time.localtime(time.time()))
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter('./savevideo/'+name+'.avi',fourcc, 30.0, (640,480))
    def threadStart(self):
        threadone=threading.Thread(target=self.mythread,args=())
        threadone.start()
      
    def mythread(self):
        global data_queue,out_queue
        print('start thread ->>> efficientdet ')
        while True:
            self.t=time.time()
            while(data_queue.qsize()<2):
                time.sleep(0.01)
                if time.time()-self.t>10:
                    print('stop thread ->>> efficientdet ')
                    break
            out_queue.put(self.detector(data_queue.get()))
                            ##################
                            ####detector######
                            ##################
    def  detector(self,im):
        ori_imgs, framed_imgs, framed_metas = sirxiapreprocess(im, max_size=self.input_size)
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
        x = x.to(torch.float32 if not self.use_float16 else torch.float16).permute(0, 3, 1, 2)
        with torch.no_grad():
            features, regression, classification, anchors = self.model(x)
            regressBoxes = BBoxTransform()
            clipBoxes = ClipBoxes()
            out = postprocess(x,
                              anchors, regression, classification,
                              regressBoxes, clipBoxes,
                              self.threshold, self.iou_threshold)
            out = invert_affine(framed_metas, out)
            bbox_list,c_list=self.update(out, ori_imgs)
            #print(bbox_list,c_list)
            #identities,bbox_xyxy = DeepSort.deep_sort(np.asarray(bbox_list),np.asarray(c_list),im)
            identities,dths,color = self.pca.efficientdet_compute_pca(bbox_list,c_list,im)        
            im =self.display(bbox_list,c_list ,im,identities,dths,color) 
            #for bbox,id_ in zip(bbox_list,identities):
                #self.bbox_save.append(bbox)
                #self.id_save.append(id_)
            if self.test:
                self.save[self.frame]=[identities,bbox_list]
            self.out.write(im)
            self.t=time.time()
            cv2.waitKey(1)
            self.frame+=1
            #return identities,bbox_xyxy,c_list
            #return bbox_list,c_list ,identities
    #过滤
    def update(self,preds, imgs):
        save_list=[]
        area_list=[]
        score_list=[]
        for i in range(len(imgs)):
            if len(preds[i]['rois']) == 0:
                continue
            for j in range(len(preds[i]['rois'])):
                if preds[i]['class_ids'][j]==0:#person
                    score = float(preds[i]['scores'][j])#百分比
                    H,W=imgs[i].shape[:2]
                    (x1, y1, x2, y2) = preds[i]['rois'][j].astype(np.int)
                    area = (x2-x1)*(y2-y1)
                    if  (area/H*W)<self.bbox_threshold or score<self.probability_threshold:
                        continue
                    x,y,w,h=(x1+x2)/2,(y1+y2)/2,abs(x1-x2),abs(y1-y2)
                    area_list.append(area)
                    save_list.append([x,y,w,h]) 
                    score_list.append(score)
        bbox_list,c_list=[],[]
        if len(area_list)>self.humanNum:
            
            for i in range(self.humanNum):
                one=save_list[area_list.index(max(area_list))]
                two=score_list[area_list.index(max(area_list))]
                bbox_list.append(copy.copy(one))
                c_list.append(copy.copy(two))
                save_list.remove(one)
                score_list.remove(two)
                area_list.remove(max(area_list))
                 
        else:
            for i in range(len(area_list)):
                one=save_list[area_list.index(max(area_list))]
                two=score_list[area_list.index(max(area_list))]
                bbox_list.append(copy.copy(one))
                c_list.append(copy.copy(two))
                save_list.remove(one)
                score_list.remove(two)
                area_list.remove(max(area_list))
        return bbox_list,c_list
    
    def display(self,bbox_list,c_list,im,identities,dths,color):
        #print(len(out_list))
        for bbox,c,id_,dth,color_ in zip(bbox_list,c_list,identities,dths,color):
            #(x1, y1, x2, y2) = bbox
            (x, y, w, h) = bbox
            (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
            cv2.rectangle(im, (x1, y1), (x2, y2), (0, color_, 255-color_), 2)
            #cv2.putText(im, ('%.2f'%c), (x2+10, y1+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 250,0), 1)
            cv2.putText(im, ('%s'%id_), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, color_, 255-color_), 1.5)
            cv2.putText(im, ('%.2f'%dth), (x2, y1+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, color_, 255-color_), 1)
        fps = 1/(time.time()-self.t)
        cv2.putText(im, ('%.2f'%fps), (590, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.imshow('efficientdet', im)
        return im

In [209]:
from torchvision.models.segmentation import fcn_resnet101
from PIL import Image
from torchvision import transforms
from scipy.optimize import linear_sum_assignment
class PCA:
    def __init__(self,dir_path='./test_colors/train',NUM_EIGEN_FACES = 64):
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False}}
        self.NUM_EIGEN_FACES = NUM_EIGEN_FACES
        self.images = self.readImages(dir_path)
        self.data = self.createDataMatrix(self.images)
        #self.data = torch.from_numpy(self.data).cuda()
        #torch.pca_lowrank(A, q=None, center=True, niter=2)
        #self.mean, self.eigenVectors = torch.pca_lowrank(self.data, q=None, center=True, niter= self.NUM_EIGEN_FACES)
        #self.means, self.eigenVectorses=[],[]
        #print('the alldata %d'%len(self.eigenVectors))
        #self.init_pca(basepath)
        #self.notebook={'000':{'frame':[],'lib':[]}}
        self.frame=0
        self.color_old=0
        self.flag_frame=[]
        self.flag_line=[]
        self.dth = 0.5#相似度距离0.6  / 欧氏距离25
        self.Forget=30  #状态切换帧
        self.testsave={}
        self.models_mean = './models/mean.npy'
        self.models_eigenVector = './models/eigenVector.npy'
        #self.meta = threading.Lock()
        if os.path.exists(self.models_mean)==False and os.path.exists(self.models_eigenVector)==False:
            self.mean, self.eigenVectors = cv2.PCACompute(self.data, mean=None, maxComponents=self.NUM_EIGEN_FACES)
            np.save(self.models_mean,self.mean)
            np.save(self.models_eigenVector,self.eigenVectors)
        else:
            self.mean=np.load(self.models_mean)
            self.eigenVectors=np.load(self.models_eigenVector)
        #卡尔曼滤波
        self.kalman=[]
    def createDataMatrix(self,images):
        numImages = len(images)
        sz = images[0].shape
        data = np.zeros((numImages, sz[0] * sz[1] * sz[2]), dtype=np.float32)
        for i in range(0, numImages):
            image = images[i].flatten()
            data[i,:] = image
        #print("createData ok")
        return data

    def readImages(self,path):
        print("Reading images from " + path, end="...")
        # Create array of array of images.
        images = []
        # List all files in the directory and read points from text files one by one
        for name in glob.glob(path+'/*'):
            for imagePath in glob.glob(name+'/*.jpg'):
                    # Add to array of images
                    im = cv2.imread(imagePath)
                    im = cv2.resize(im,(64,128))
                    if im is None :
                        print("image:{} not read properly".format(imagePath))
                    else :
                        # Convert image to floating point
                        im = np.float32(im)/255.0
                        # Add image to list
                        images.append(im)
                        # Flip image 
                        imFlip = cv2.flip(im, 1);
                        # Append flipped image
                        #images.append(imFlip)
        numImages = int(len(images))
        # Exit if no image found
        if numImages == 0 :
            print("No images found")
            sys.exit(0)
        print(str(numImages) + " files read.")
        return images
#*
#*
#*      算法集合
#*
#*
    def compute_cos(self,x,y):
        x_,y_=x.flatten(),y.flatten()
        dist =1- abs(np.dot(x_,y_)/(np.linalg.norm(x_)*np.linalg.norm(y_)))      
        return abs(dist)
    
    def compute_dis(self,x,y):
         return np.linalg.norm( x - y )
    
    def compute_Maha(self,x,y):
        X=np.vstack([x,y])
        XT=X.T
        S=np.cov(X)   #两个维度之间协方差矩阵
        SI = np.linalg.inv(S) #协方差矩阵的逆矩阵
        #马氏距离计算两个样本之间的距离，此处共有10个样本，两两组合，共有45个距离。
        n=XT.shape[0]
        d1=[]
        for i in range(0,n):
            for j in range(i+1,n):
                delta=XT[i]-XT[j]
                d=np.sqrt(np.dot(np.dot(delta,SI),delta.T))
                d1.append(d)
        return d1
    
    def computer_Hungary(self,task_matrix):
        b = task_matrix.copy()
        # 行和列减0
        for i in range(len(b)):
            row_min = np.min(b[i])
            for j in range(len(b[i])):
                b[i][j] -= row_min
        for i in range(len(b[0])):
            col_min = np.min(b[:, i])
            for j in range(len(b)):
                b[j][i] -= col_min
        line_count = 0
        # 线数目小于矩阵长度时，进行循环
        while (line_count < len(b)):
            line_count = 0
            row_zero_count = []
            col_zero_count = []
            for i in range(len(b)):
                row_zero_count.append(np.sum(b[i] == 0))
            for i in range(len(b[0])):
                col_zero_count.append((np.sum(b[:, i] == 0)))
            # 划线的顺序（分行或列）
            line_order = []
            row_or_col = []
            for i in range(len(b[0]), 0, -1):
                while (i in row_zero_count):
                    line_order.append(row_zero_count.index(i))
                    row_or_col.append(0)
                    row_zero_count[row_zero_count.index(i)] = 0
                while (i in col_zero_count):
                    line_order.append(col_zero_count.index(i))
                    row_or_col.append(1)
                    col_zero_count[col_zero_count.index(i)] = 0
            # 画线覆盖0，并得到行减最小值，列加最小值后的矩阵
            delete_count_of_row = []
            delete_count_of_rol = []
            row_and_col = [i for i in range(len(b))]
            for i in range(len(line_order)):
                if row_or_col[i] == 0:
                    delete_count_of_row.append(line_order[i])
                else:
                    delete_count_of_rol.append(line_order[i])
                c = np.delete(b, delete_count_of_row, axis=0)
                c = np.delete(c, delete_count_of_rol, axis=1)
                line_count = len(delete_count_of_row) + len(delete_count_of_rol)
                # 线数目等于矩阵长度时，跳出
                if line_count == len(b):
                    break
                # 判断是否画线覆盖所有0，若覆盖，进行加减操作
                if 0 not in c:
                    row_sub = list(set(row_and_col) - set(delete_count_of_row))
                    min_value = np.min(c)
                    for i in row_sub:
                        b[i] = b[i] - min_value
                    for i in delete_count_of_rol:
                        b[:, i] = b[:, i] + min_value
                    break
        row_ind, col_ind = linear_sum_assignment(b)
        min_cost = task_matrix[row_ind, col_ind].sum()
        best_solution = list(task_matrix[row_ind, col_ind])
        return  best_solution
     #计算两点近距离公式 xyxy
    def distEclud(self,veA,vecA,veB,vecB):
        lossA=veB-veA
        lossB=vecB-vecA
        return math.sqrt(pow(lossA,2)+pow(lossB,2))
#*
#*
#*      功能函数
#*
#*
    def clear_id(self):
        self.save={'000':{'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False}}   
        
    def kalman_reid(self,bbox):
        (x,y,w,h)=bbox
        list_kalman_distance,list_kalman_id=[],[]
        for k,v in list(self.save.items()):
            if k!='000'and len(v['lib'])>10:
                list_kalman_distance.append(self.distEclud(x,y,v['kalman'][0],v['kalman'][1]))
                list_kalman_id.append(k)
        if len(list_kalman_id)>0:
            id_ = list_kalman_id[list_kalman_distance.index(min(list_kalman_distance))]
            #self.save[id_]['cross']=True
        else:
            id_ = ' '
        return id_
    def manage_lib(self,frame):
        for k,v in list(self.save.items()):
            if k=='000':#or v['passage']==False:
                continue
            lib = v['lib']
            frame_lib=v['frame']
            if frame-frame_lib>self.Forget:
                self.save[k]['frame']=frame
                #self.save[k]['lib'].pop(0)
                if len(lib)<10:
                    self.dele_lib_id(k)
                if  k not in self.save:
                    continue
                if self.save[k]['passage']==True:
                    center =self.save[k]['center']
                    for lib_ in self.save[k]['lib']:
                        center=center*0.99+lib_*0.01
                    #self.save[k]['lib'].pop(0)
                    self.passage_off(k,center)
                    #self.flag_frame=[]
                    
    def paassag_on(self,feature,bbox,frame,id_):
        self.save[id_]['center']=feature
        self.save[id_]['bbox']=bbox
        self.save[id_]['lib'].append(feature)
        self.save[id_]['frame']=frame
        self.save[id_]['passage']=True
        print('%s passage is True'%id_)
        
    def passage_off(self,id_,center):
        self.save[id_]['center']=center
        self.save[id_]['passage']=False
        print('%s passage is False'%id_)
        
    def update_frame(self,id_,frame):
        self.save[id_]['frame']=frame
        
    def dele_lib_id(self,id_):
        del self.save[id_]
        print('the id %s Remove'%id_)
        
    def add_lib(self,feature,bbox,frame,passage=False):
        save={'center':[],'bbox':[],'lib':[],'frame':0,'passage':False,'kalman':[],'cross':False}
        for n in  self.save.keys():
            id_=n
        self.kalman.append(KalmanFilter(bbox))
        self.kalman[(int(id_))].update(bbox)
        x,y=self.kalman[(int(id_))].get()
        
        save['kalman']=[x,y]
        self.id_new=('%03d'%(int(id_)+1))
        save['center']=feature
        save['bbox']=bbox
        save['lib']=[feature]
        save['frame']=frame
        save['passage']=passage
        self.save[self.id_new]=save
        print('find id %s'%self.id_new)
        return self.id_new
    
    def add_id(self,feature,bbox,frame):   
        #print(len(self.save))
        if frame!=0 and len(self.save)!=1:
            id_ = self.check_lib_passage(feature)
            #self.id_new = self.check_lib(feature)
            if id_ == ' ':
                self.flag_line=[]
                if self.id_new in self.save:
                    if frame-self.save[self.id_new]['frame']>1:
                        self.flag_frame=[]
                else:
                    self.flag_frame=[]
                if len(self.flag_frame)==0:
                    self.id_new = self.add_lib(feature,bbox,frame)
                self.flag_frame.append(frame)
                self.save[self.id_new]['frame']=frame
                #self.update_frame
                if len(self.flag_frame)>=5:
                    if np.mean(self.flag_frame)==self.flag_frame[2]:
                        self.paassag_on(feature,bbox,frame,self.id_new)
                        self.flag_frame=[]
                    else:
                        self.dele_lib_id(self.id_new)
                        self.flag_frame=[]
            else:
                #if frame - self.save[id_]['frame'] >3:
                self.flag_frame=[]
                self.flag_line.append(frame)
                self.save[id_]['frame']=frame
                #self.update_frame
                if len(self.flag_line)>=3:
                    if np.mean(self.flag_line)==self.flag_line[1]:
                        self.paassag_on(feature,bbox,frame,id_)
                        self.flag_line=[]
                    else:
                        self.flag_line=[]
                return id_
        else:
            self.id_new = self.add_lib(feature,bbox,frame,passage=True)
        return self.id_new
#*
#*      检测集合
#*                        
    def check_frame_id(self,id_,feature,frame):
        list_dis=[]
        for k,v in list(self.save.items()):
            if k==id_:
                if frame-(v['frame'])>=10:
                        if k==id_:
                            for lib_ in v['lib']:
                                list_dis.append(self.compute_cos(lib_,feature))
                            if min(list_dis)<self.dth:
                                return True
                            else:
                                return False
                else:
                    return True
        return False
    #检查除id_me外是否与数据库里的bbox重叠
    def check_bboxs(self,bbox_list,bbox):    
        (xc, yc, wc, hc) =bbox
        list_id,list_x=[],[]
        ret=True
        if len(bbox_list)!=1:
        #if (5<(xc-(wc*0.5)) and(xc+(wc*0.5))<(im_w-10)) or (5<(yc-(hc*0.5)) and (yc+(hc*0.5))<(im_h-10)):
            for i in range(len(bbox_list)):
                for j in range(i+1,len(bbox_list)):
                    (xa, ya, wa, ha) = bbox_list[i]
                    (xb, yb, wb, hb) = bbox_list[j] 
                    distance = self.distEclud(xa,ya,xb,yb)
                    if (distance/(wa+wb))>=0.5:
                        continue
                    if (distance/(wa+wb))<0.5:
                        if xc==xa or xc==xb:#是不是因为重叠导致的
                            ret=False
        return ret
    def check_bboxs_cross(self,bboxs,bbox):    
        ret=True
        (xa, ya, wa, ha)=bbox
        cross_bboxs=copy.copy(bboxs)
        cross_bboxs.remove(bbox)
        for bbox_ in cross_bboxs:
            (xb, yb, wb, hb) = bbox_ 
            distance = self.distEclud(xa,ya,xb,yb)
            if (distance/(wa+wb))>=0.5:
                continue
            if (distance/(wa+wb))<0.5:
                ret=False
        return ret
    def check_bboxs_overlap(self,bboxs,bbox):
        ret=True
        over_bboxs=copy.copy(bboxs)
        over_bboxs.remove(bbox)
        for bbox in zip(over_bboxs):
            (xc, yc, wc, hc)=bbox
            for k,v in list(self.save.items()):
                if v['passage']==True and k!='000'and len(v['lib'])>10:
                    (xl, yl, wl, hl) = v['bbox']
                    distance = self.distEclud(xc,yc,xl,yl)
                    if (distance/(wc+wl))>=0.5:
                        continue
                    if (distance/(wc+wl))<0.5:  
                        ret=False
        return ret
    def check_lib_add(self,featureVector,min_distance_id,bbox):
        for k,v in list(self.save.items()):
            list_dis=[]
            if  k==min_distance_id:
                n=v['center']
                lib = v['lib']
                #跟新kalman 轨迹预测
                kalman_id=int(k)-1
                self.kalman[kalman_id].update(bbox)
                x,y =self.kalman[kalman_id].get()
                self.save[k]['kalman']=[x,y]
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                if len(self.save[min_distance_id]['lib'])>511:
                    num = list_dis.index(min(list_dis))
                    self.save[min_distance_id]['lib'].pop(num)
                    #self.save[min_distance_id]['lib'].pop(0)
                if len(list_dis)>0 and np.mean(list_dis)>self.dth:
                    self.save[min_distance_id]['lib'].append(featureVector)
                    print('\r%s lib len :%d'%(min_distance_id,len(lib)),end='')
                    #跟新kalman bbox
                    num=int(k)-1
                    self.kalman[num].update(bbox)
                break
        return int(len(lib)/2)
    #检查就绪的
    def check_lib(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k=='000'or v['passage']==False :
                continue
            lib = v['lib']
            for lib_ in lib:
                list_dis.append(self.compute_cos(lib_,featureVector))
            list_id.append(k)
            list_mean.append(min(list_dis)) 
        if len(list_mean)>0 and min(list_mean) <self.dth:
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            return ' '   
    #检查未就绪
    def check_lib_passage(self,featureVector):
        list_id,list_mean=[],[]
        for k,v in list(self.save.items()):
            list_dis=[]
            if k!='000' and v['passage']==False and len(v['lib'])>10:
                lib = v['lib']
                for lib_ in lib:
                    list_dis.append(self.compute_cos(lib_,featureVector))
                list_id.append(k)
                list_mean.append(min(list_dis)) 
        if len(list_mean)>0 and min(list_mean) <self.dth:
            return list_id[list_mean.index(min(list_mean))] 
        else:
            #print( min(list_mean))
            return ' '    
   
    #def efficientdet_compute_pca(self,bbox_list,confidences,im):
    def efficientdet_compute_pca(self,bbox_list,confidences,im):
        #
        # * self.notebook 记录发现新的图片的W
        # * 保存数据的格式 self.save={'id':{'center':w,'LIB':[w1..wn]}}
        #
        color=[]
        identities=[]
        dths=[]
        #维护LIB
        self.manage_lib(self.frame)
        if bbox_list!=[]:
            list_id,list_feature,list_distance,list_center=[],[],[],[]
            user_bbox_list=copy.copy(bbox_list)
            for bbox,c in zip(user_bbox_list,confidences):
                (x, y, w, h) = bbox
                (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
                #(x,y,w,h)=(x1+x2)*0.5,(y1+y2)*0.5,abs(x1-x2),abs(y1-y2)
                im_=im[y1:y2,x1:x2]
                im_ = cv2.resize(im_,(64,128))
                featureVector = self.compute_feature(im_)
                #交叉判断
                #print(bbox_list)
                ret_bbox_cross = self.check_bboxs_cross(user_bbox_list,bbox)
                #ret_bbox_overlap=self.check_bboxs_overlap(bbox_list,bbox)
                #有交叉则使用kalman跟踪
                if ret_bbox_cross ==False :#or ret_bbox_overlap==False:
                    bbox_list.remove(bbox)
                    id_ = self.kalman_reid(bbox) 
                    print(id_)
                    if id_==' ':
                        id_ = self.add_id(featureVector,bbox,self.frame)
                        identities.append(id_)
                        dths.append(c)
                        color.append(0)
                    else:
                        self.save[id_]['bbox'] =bbox
                        self.save[id_]['frame'] =self.frame
                        dths.append(c)
                        identities.append(id_)
                        color_=self.check_lib_add(featureVector,id_,bbox)
                        color.append(color_)
                #PCA算法
                else:
                    distance=0
                #所有图片与LIB中passage为TRUE的计算距离
                    list_distance_buffer,list_id_buffer,list_feature_buffer,list_center_buffer=[],[],[],[]
                    for k,v in list(self.save.items()):
                        if k=='000':
                            continue
                        if v['passage']==True:
                            n=v['center']
                            #kalman_xy=v['kalman']
                            #distance_kalman = self.distEclud(kalman_xy[0],kalman_xy[1],bbox[0],bbox[1])
                            #print(distance_kalman)
                            distance = self.compute_cos(n,featureVector)
                            #distance =+distance_feature*0.95+distance_kalman*0.005
                            list_distance_buffer.append(distance)
                            list_id_buffer.append(k)
                            list_feature_buffer.append(featureVector)
                            list_center_buffer.append(n) 

                    if len(list_distance_buffer)>0 :#and min(list_distance_buffer)<self.dth:
                        list_distance.append(list_distance_buffer)
                        list_id.append(list_id_buffer)
                        list_feature.append(list_feature_buffer)
                        list_center.append(list_center_buffer)
                    else:
                        #ret = self.check_bboxs(bbox_list,bbox) 
                        #if ret ==True:
                        id_ = self.add_id(featureVector,bbox,self.frame)
                        identities.append(id_)
                        dths.append(c)
                        color.append(0) 
            #匈牙利算法
            list_bbox_loss,list_feature_loss=[],[]
            if list_distance!=[]:
                    kalman_center,kalman_feature,kalman_id,kalman_bbox=[],[],[],[]
                    kalman_bbox=bbox_list
                    #需要id的数量要小于等于就绪id的数量
                    #print(len(bbox_list),len(list_distance[0]))
                    if len(bbox_list)>len(list_distance[0]):
                        min_distance=[]
                        #print(list_distance,bbox_list)
                        for id_distanc in list_distance:
                            min_distance.append(min(id_distanc))
                        for i in range((len(bbox_list)-len(list_distance[0]))):
                            num=min_distance.index(max(min_distance))
                            min_distance.pop(num)
                            list_distance.pop(num)
                            list_id.pop(num)
                            list_center.pop(num)  
                            list_feature_loss.append(list_feature.pop(num))
                            list_bbox_loss.append(kalman_bbox.pop(num))
                            #print(list_bbox_loss,list_feature_loss)
                            confidences.pop(num)
                    if list_distance==[]:
                        return identities,dths,color
                    np_distance=np.asarray(list_distance)
                    #print(np_distance)
                    #print(self.save.keys())
                    list_Hungary = self.computer_Hungary(np_distance)
                    #print(list_Hungary)
                    list_id_,list_center_,list_feature_=[],[],[]
                    for id_s,center_s,feature_s in zip(list_id,list_center,list_feature):
                        for id_,center_,feature_ in zip(id_s,center_s,feature_s):
                            list_id_.append(id_)
                            list_center_.append(center_)
                            list_feature_.append(feature_)
                    for number in list_Hungary:
                        #number = number*0.00001               
                        min_distance = np_distance.flatten().tolist().index(number)
                        kalman_id.append(list_id_[min_distance])
                        kalman_center.append(list_center_[min_distance])
                        kalman_feature.append(list_feature_[min_distance])

                    #检查重新回来的id的frame
                    #ret_id_frame = self.check_frame_id(min_distance_id,min_distance_feature,self.frame)
                    #print(kalman_bbox,kalman_center,kalman_feature,kalman_id,confidences)
                    #ret_bbox_cross = self.check_bboxs_cross(kalman_bbox,kalman_id)
                    #if ret_bbox_cross==False :
                        #kalman_bbox=self.kalman_reid(kalman_bbox,kalman_id)
                    for bbox,conter,feature,id_,c in zip(kalman_bbox,kalman_center,kalman_feature,kalman_id,confidences):
                         #检查除id_me外是否与数据库里的bbox重叠
                        center_ = (conter*0.5)+(feature*0.5)
                        self.save[id_]['center'] =center_
                        self.save[id_]['bbox'] =bbox
                        self.save[id_]['frame'] =self.frame
                        color_=self.check_lib_add(feature,id_,bbox)
                        color.append(color_)
                        dths.append(c)
                        identities.append(id_)
                    #没有ID,重新add_id    
                    #print(list_bbox_loss,list_feature_loss,confidences)
                    for bbox,featureVector,c in zip(list_bbox_loss,list_feature_loss,confidences):
                        #print(bbox,featureVector,c)
                        id_ = self.add_id(featureVector[0],bbox,self.frame)
                        identities.append(id_)
                        dths.append(c)
                        color.append(0)  
        self.frame+=1
        return identities,dths,color
    #输入图片计算，输出Feature
    def compute_feature(self,im):
        im_ = im
        size=im_.shape
        im_ = np.float32(im_)/255.0
        Fim = im_.flatten()
        Fmean = self.mean.reshape(Fim.shape)
        Fdf = Fim-Fmean
        Fdf= Fdf.reshape(24576,1)
        W=[]
        for i in range(len(self.eigenVectors)):
            E = self.eigenVectors[i,:].reshape(1,24576)
            W.append(np.dot(E,Fdf).flatten())
        W = np.asarray(W)
        return W
    #每一帧图片的ims
    def compute_pca(self,ims):
        self.classes=['A','B','C','D']
        #num=0
        #if  self.frame==0:
        print('initfeature')
        #self.flag==True
        featureVector = self.compute_feature(ims[0])
        self.testsave[self.classes[0]]=featureVector
            #self.frame+=1  
        for k,v in self.testsave.items():
            list_,list_W,list_w0=[],[],[]
            list_mashi1,list_mashi2=[],[]
            n = np.asarray(v)
            for im in ims:
                #im=self.fill_im(im)
                #im=cv2.resize(im,(64,128))
                featureVector = self.compute_feature(im)
                list_W.append(featureVector)
                n_ = n.reshape((128,2))
                feature_ = featureVector.reshape((128,2))
                list_.append(self.compute_Maha(n_,feature_))
                #self.frame+=1
                n =np.asarray(featureVector)*0.5+n*0.5#根据三比七的比重进行调节
                #self.testsave[self.classes[0]]=n
                list_w0.append(n)
        #print('\r %d'%self.frame,end ='')
        for W in list_W:
            self.createNewFace(W,self.frame)
            self.frame+=1
        return list_ , list_W ,list_w0
    def createNewFace(self,W,num):
        # Start with the mean image
        output = self.mean.reshape(64,128,3)
        w=W.tolist()
        # Add the eigen faces with the weights
        for i in range(0, self.NUM_EIGEN_FACES):
            '''
            OpenCV does not allow slider values to be negative. 
            So we use weight = sliderValue - MAX_SLIDER_VALUE / 2
            ''' 
            #sliderValues[i] = cv2.getTrackbarPos("Weight" + str(i), "Trackbars");
            weight = w[i]
            e = self.eigenVectors[i].reshape(64,128,3)
            output = np.add(output, e * weight)

        # Display Result at 2x size
        #output = cv2.resize(output, (0,0), fx=2, fy=2)
        output=output*255.0
        cv2.imshow("Result", output)
        name=('%06d.jpg'%num)
        cv2.imwrite('./walkingworkspace/output/'+name,output)
        cv2.waitKey(1)
    def pca_class_dataset(self,im,name,output_path):
        identities = os.path.basename(name)
        self.frame+=1
        #testimage = self.im_creat(testimage)
        im_ = im
        #sz = self.images[0].shape
        size=im_.shape
        im_ = np.float32(im_)/255.0
        Fim = im_.flatten()
        Fmean = self.mean.reshape(Fim.shape)
        Fdf = Fim-Fmean
        Fdf= Fdf.reshape(24576,1)
        W=[]
        for i in range(len(self.eigenVectors)):
            E = self.eigenVectors[i,:].reshape(1,24576)
            #print(self.eigenVectors[i,:].reshape(1,24576).T.shape)
            W.append(np.dot(E,Fdf).flatten())
        W = np.asarray(W)
        #print(W)
        #cv2.imshow('build',build[0:])
        name = ('/%s'%identities)
        name = name.replace('jpg','npy')
        np.save(output_path+name,W)
        #cv2.imwrite(name+'.jpg',im)
        cv2.waitKey(1)
    def test_add_lib(self,im_,id_):
        list_distance,list_feature=[],[]
        featureVector = self.compute_feature(im_)
        for k,v in list(self.save.items()):
            if k=='000'and  k!=id_:
                continue
            if v['passage']==True:
                n=v['center']
                distance = self.compute_cos(n,featureVector)
                if distance <self.dth:
                    list_distance.append(distance)
                    list_feature.append(featureVector)
        if len(list_distance)!=0:
                min_distance_feature = list_feature[list_distance.index(min(list_distance))]
                center = (min_distance_feature*0.5)+(featureVector*0.5)
                self.save[id_]['center'] =center
                self.check_lib_add(min_distance_feature,id_,bbox)
                                        
    def test(self,path):
        list_val1,list_val2,list_test1,list_test2,list_test1_loss,list_test2_loss=[],[],[],[],[],[]
        for name in glob.glob(path+'/*'):
            if name.find('xia001')!=-1:
                for im_name1 in glob.glob(name+'/*.jpg'):
                    list_val1.append(cv2.imread(im_name1))
            if name.find('li002')!=-1:
                for im_name2 in glob.glob(name+'/*.jpg'):
                    list_val2.append(cv2.imread(im_name2))
                    
        featureVector = self.compute_feature(list_val1[0])
        self.add_lib(featureVector,[],1,passage=True)
        featureVector = self.compute_feature(list_val2[0])
        self.add_lib(featureVector,[],1,passage=True)
        for im_ in list_val1:
            cv2.imshow('in',im_)
            self.test_add_lib(im_,'001')
            self.kalman.append(KalmanFilter())
            cv2.waitKey(1)
        for im_ in list_val2:
            cv2.imshow('in',im_)
            self.test_add_lib(im_,'002') 
            self.kalman.append(KalmanFilter())
            cv2.waitKey(1)
            
        center =self.save['001']['lib'][0]
        for lib_ in self.save['001']['lib']:
            center=center*0.5+lib_*0.5
        #self.save[k]['lib'].pop(0)
        self.passage_off('001',center)
        
        center =self.save['002']['lib'][0]
        for lib_ in self.save['002']['lib']:
            center=center*0.5+lib_*0.5
        #self.save[k]['lib'].pop(0)
        self.passage_off('002',center)

efficientdet + PCA

In [210]:
HumanMaxNumber=4
size=(640,480)
E = efficientdet(0 ,640,HumanMaxNumber,dir_path='./walkingworkspace/train2020',NUM_EIGEN_FACES = 256,test=False)
#E = efficientdet(0 ,512,HumanMaxNumber,dir_path='./datasets/multi-query',NUM_EIGEN_FACES = 256)
#从新构建models
#E.init_PCA()

Reading images from ./walkingworkspace/train2020...1982 files read.


In [208]:
E.init_video()
#videopath='./V/liandxia.mp4'
#videopath='./testvideo/two_rotation.mp4'
videopath='./testvideo/test_kelman.mp4'
#videopath='./walkingvideo/two_rotationwalking.mp4'
#videopath=0
video = cv2.VideoCapture(videopath)
video_fps = video.get(7)
print(video_fps)
num=0
for i in range(int(video_fps-1)):
#t=time.time()

#while True:
#    if (time.time()-t)>=5:
#        if (time.time()-t)>=300:
#            break 
        ret,im = video.read()
        #im_=cv2.flip(im,0)
        if ret:
            if num%1==0:
                E.detector(cv2.resize(im,size))
        num+=1
cv2.destroyAllWindows() 
E.out.release()
video.release()    

335.0
find id 001
find id 002
001 lib len :14001
001 lib len :15001
001 lib len :16001
001 lib len :17001
001 lib len :18001
001 lib len :19001
001 lib len :20001
001 lib len :21001
001 lib len :22001
001 lib len :23001
002 lib len :15002
002 lib len :16001
001 lib len :25002
002 lib len :17001
002 lib len :74001
001 lib len :81001
002 lib len :89002
002 lib len :90001
001 lib len :83002
002 lib len :91001
002 lib len :109

KeyboardInterrupt: 

# PCATEST


In [ ]:
HumanMaxNumber=4
size=(640,480)
E = efficientdet(0 ,640,HumanMaxNumber,dir_path='./walkingworkspace/train2020',NUM_EIGEN_FACES = 256,test=True)
#E.init_video()
videopath='./testvideo/test_kelman.mp4'
#videopath=0
video = cv2.VideoCapture(videopath)
video_fps = video.get(7)
print(video_fps)
num=0
for i in range(int(video_fps-1)):
#t=time.time()
#while True:
#    if (time.time()-t)>=5:
#        if (time.time()-t)>=300:
#            break 
        ret,im = video.read()
        #im_=cv2.flip(im,0)
        if ret:
            if num%1==0:
                E.detector(cv2.resize(im,size))
        num+=1
cv2.destroyAllWindows() 
E.out.release()
video.release()    

In [ ]:

np.save('./kalman.npy',[E.save])
print(E.save)        

In [ ]:

num_test_002_true,num_test_002_false,num_test_001_true,num_test_001_false=0,0,0,0
path = './walkingworkspace/test2020'
list_label,list_li,list_xia,list_other=[],[],[],[]
for name in glob.glob(path+'/LI/*.jpg'):
    list_li.append(os.path.basename(name))
for name in glob.glob(path+'/XI/*.jpg'):
    list_xia.append(os.path.basename(name))
for name in glob.glob(path+'/other/*.jpg'):
    list_other.append(os.path.basename(name))
    
for name in glob.glob('./walkingworkspace/action/*.jpg'):
    name = os.path.basename(name) 
    if name in list_other:
        list_label.append('003')
    if name in list_xia:
        list_label.append('001')   
    if name in list_li:
        list_label.append('002')

list_label=list_label[(len(list_label)-len(E.label_test)):len(list_label)]
print(len(E.label_test),len(list_label))  

In [ ]:
for test_label,label in  zip(E.label_test,list_label):
    if label=='001':
        if test_label==label:
            num_test_001_true+=1
        else:
            num_test_001_false+=1
    if label=='002':
        if test_label==label:
            num_test_002_true+=1
        else:
            num_test_002_false+=1
print('the 001 C:%.2f'%(num_test_001_true/(num_test_001_true+num_test_001_false)))
print('the 002 C:%.2f'%(num_test_002_true/(num_test_002_true+num_test_002_false)))

# PCA testdata rebuild

In [ ]:
#HumanMaxNumber=2
#size=(512,384)
#E = efficientdet(0 ,512,HumanMaxNumber)
#ids_bboxes = {}
pca=PCA(dir_path='./walkingworkspace/train2020',NUM_EIGEN_FACES = 256)

In [ ]:
#E.threadStart()
#videopath='./testvideo/mc1.mp4'
im_path='./walkingworkspace/action/'
list_im=[]
for im_name in glob.glob(im_path+'*.jpg'):
    list_im.append(im_name.replace('\\','/'))
list_W=[]
list_=[]

print(len(list_im))
list_im1,list_im2,list_im3,list_im4=[],[],[],[]
for im_name in list_im:  
    #print(im_name)
    im = cv2.imread(im_name)
    im=cv2.resize(im,(64,128))
    #if im_name.find('A')!=-1: 
    list_im1.append(im)
    #if im_name.find('B')!=-1:
    #    list_im2.append(im)
    #if im_name.find('C')!=-1:
        #list_im3.append(im)
    #if im_name.find('D')!=-1:
        #list_im4.append(im)
distance1,w1,wcenter1= pca.compute_pca(list_im1)
#pca=PCA(dir_path='./test_colors/train',basepath = './test_colors/source',NUM_EIGEN_FACES = 64)
#distance2,w2,wcenter2= pca.compute_pca(list_im2)
#data3= pca.compute_pca(list_im3)
#data4= pca.compute_pca(list_im4)
print(distance1)
#print(len(distance2))
#print(data2)
#    X,Y,Z=data
'''if im_name.find('A')!=-1:
    set_A_x.append(X)
    set_A_y.append(Y)
    set_A_z.append(Z)
if im_name.find('B')!=-1:
    set_B_x.append(X)
    set_B_y.append(Y)
    set_B_z.append(Z)'''
'''if im_name.find('O')!=-1:
    set_O_x.append(X)
    set_O_y.append(Y)
    set_O_z.append(Z)
if im_name.find('C')!=-1:
    set_C_x.append(X)
    set_C_y.append(Y)
    set_C_z.append(Z)
if im_name.find('E')!=-1:
    set_E_x.append(X)
    set_E_y.append(Y)
    set_E_z.append(Z)  ''' 
#    list_W.append(W)
    #pca.pca_class_dataset(im,im_name,output_path)
#pca.test_meandyou()
cv2.destroyAllWindows() 
#print(len(set_A_x),len(set_A_y))

In [ ]:

#data1.remove(max(data1))
#data1.remove(min(data1))
#data2.remove(max(data2))
#data2.remove(min(data2))
max_jiawei = max(data1)
max_xiaobo = max(data2)
print(max_jiawei,max_xiaobo)
max_jiawei,max_xiaobo = float(max_jiawei),float(max_xiaobo)
print((max_jiawei+max_xiaobo)/2)

In [ ]:
import matplotlib.pyplot as plt
plt.ylabel('I am distance')
plt.xlabel('I am id')
#plt.legend([L1,L2],['A','B'],loc='upper right')
x1=range(0,len(distance1))
#x2=range(0,len(distance2))
x3=range(0,50)
x4=range(0,50)
plt.plot(x1,distance1,label='A',linewidth=3,color='r',marker='o',
markerfacecolor='r',markersize=12)
plt.show()
#plt.plot(x2,distance2,label='B',linewidth=3,color='r',marker='o',
#markerfacecolor='g',markersize=12)
#plt.show()
#plt.plot(x1,data3,label='C',linewidth=3,color='r',marker='o',
#markerfacecolor='b',markersize=12)
#plt.plot(x2,data4,label='D',linewidth=3,color='r',marker='o',
#markerfacecolor='y',markersize=12)
#plt.show()

In [ ]:
#语义分割

In [ ]:

self.net = fcn_resnet101(pretrained=True,progress=False).cuda()
_ = self.net.eval()
def fcn_mask(self,im):
    im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
    input_batch = torch.tensor(im,dtype=torch.float32).cuda()
    input_batch = input_batch.permute(2,0,1)
    input_batch = 2*input_batch/255.0-1.0
    input_batch = input_batch.unsqueeze(0)
    with torch.no_grad():
        output = net(input_batch)['out'][0]
    output_predictions = output.argmax(0).cpu()
    output_predictions = np.where(output_predictions[:]==15,1,0)
    return output_predictions

def create_mask(self,im_bbox):
    im_fcn = self.fcn_mask(im_bbox)
    re = np.zeros(im_bbox.shape)*255
    #im_fcn_=np.zeros((im_fcn.shape[0],im_fcn.shape[1],3))
    re[:,:,0]=re[:,:,1]=re[:,:,2]=im_fcn
    #re = np.where(re.all()!=[0,0,0], [0,0,0], [255,255,255])
    re = re*im_bbox
    re = re.astype(np.uint8)
    cv2.imshow('mask',re)
    return re

In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure()
plt = fig.gca(projection='3d')
plt.plot(set_A_x,set_A_y,set_A_z,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='w',markersize=12)
plt.plot(set_B_x,set_B_y,set_B_z,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='b',markersize=12)
plt.plot(set_O_x,set_O_y,set_O_z,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='g',markersize=12)
plt.plot(set_C_x,set_C_y,set_C_z,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='y',markersize=12)
plt.plot(set_C_x,set_C_y,set_C_z,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='y',markersize=12)
plt.show()

分类器

test2

In [ ]:
import cv2
import glob
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset,ConcatDataset
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
device = torch.device("cuda:0,1" if torch.cuda.is_available() else "cpu")

In [ ]:
class DataLoader(Dataset):
    def __init__(self, fpath_peoject=[]):
        
        self.labels,self.images = [],[]
        for i in fpath_peoject:
            j = i[1]
            np_im = np.load(i[0])
            self.labels.append(j)
            np_im = np_im.flatten()
            #print(np_im.shape)
            self.images.append(np_im)
                
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return (self.images[idx],self.labels[idx])
classes=['A','B']
def get_data(path):
    list_train,list_val=[],[]
    for name in glob.glob(path+'/*.npy'):
        name_=name.replace('\\','/')
        if name_.find('A')!=-1:
            list_train.append([name_,0])
        if name_.find('B')!=-1:
            list_train.append([name_,1])
    return list_train
train = get_data('./miniclassify/train_max')  
test = get_data('./miniclassify/test_max') 


In [ ]:

mytraindataset = DataLoader(train)
mytestataset = DataLoader(test)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainloader = torch.utils.data.DataLoader(mytraindataset, batch_size=6,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(mytestataset, batch_size=4,
                                          shuffle=True, num_workers=0)
print(len(trainloader))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(100,1000)
        self.fc3 = nn.Linear(1000,2)
    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.99))
PATH='./pytorchmodel/my_max.pth'
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)
        #print(inputs,labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 6 == 0:    # print every 2000 mini-batches
            print('\r [%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss/100),end=' ')
            running_loss = 0.0

print('Finished Training')

torch.save(net.state_dict(), PATH)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images=images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

In [ ]:
net.load_state_dict(torch.load(PATH))

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)),cmap="gray")
    plt.show()
    
dataiter = iter(trainloader)
images,lables= dataiter.next()
print(type(images))
images,lables=images.to(device),lables.to(device)
print(images.shape)
outputs = net(images)
print(images.shape)
_, predicted = torch.max(outputs,1)
print(predicted)
imshow(torchvision.utils.make_grid(images.cpu()))
print('Predicted: ', ' '.join('%11s' % classes[predicted[j]]
                              for j in range(len(images))))

In [ ]:
PATH='./pytorchmodel/my_max.pth'
mynet = torch.load(PATH)
print(mynet['fc1.weight'].shape)
print(mynet['fc3.weight'].shape)
#print(mynet)
#print(type(mynet))
W = mynet['fc1.weight']
W_weight=[]
for i in range(0,100):
    #print(W[:,i].shape)
    W_weight.append(np.mean(abs(W[:,i]).cpu().numpy()))
print(W_weight)

In [ ]:
import matplotlib.pyplot as plt
x1=range(0,100)
plt.plot(x1,W_weight,label='Frist line',linewidth=3,color='r',marker='o',
markerfacecolor='blue',markersize=12)

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import cv2
import glob

# Number of EigenFaces
NUM_EIGEN_FACES = 10

# Maximum weight
MAX_SLIDER_VALUE = 10

# Directory containing images
dirName = "./pytorchdata/test"

# Read images
images=[]
for name in glob.glob(dirName+'/*.jpg'):
    #print(name)
    images.append(cv2.resize(cv2.imread(name.replace('\\','/')),(64,128)))
    cv2.imshow('0-0',cv2.imread(name.replace('\\','/')))
    cv2.waitKey(1)

# Size of images
sz = images[0].shape

# Create data matrix for PCA.
data = createDataMatrix(images)

# Compute the eigenvectors from the stack of images created
print("Calculating PCA ", end="...")
mean, eigenVectors = cv2.PCACompute(data, mean=None, maxComponents=NUM_EIGEN_FACES)
print(mean.shape, eigenVectors.shape)
print ("DONE")

averageFace = mean.reshape(sz)
print(averageFace.shape)
cv2.imshow('averageFace',averageFace)
eigenFaces = []; 

for eigenVector in eigenVectors:
    eigenFace = eigenVector.reshape(sz)
    eigenFaces.append(eigenFace)

# Create window for displaying Mean Face
cv2.namedWindow("Result", cv2.WINDOW_AUTOSIZE)

# Display result at 2x size
output = cv2.resize(averageFace, (0,0), fx=2, fy=2)
cv2.imshow("Result", output)

# Create Window for trackbars
cv2.namedWindow("Trackbars", cv2.WINDOW_AUTOSIZE)

sliderValues = []

# Create Trackbars
for i in range(0, NUM_EIGEN_FACES):
    sliderValues.append(MAX_SLIDER_VALUE/2)
    cv2.createTrackbar( "Weight" + str(i), "Trackbars", int(MAX_SLIDER_VALUE), MAX_SLIDER_VALUE, createNewFace)

# You can reset the sliders by clicking on the mean image.
cv2.setMouseCallback("Result", createNewFace);

print('''Usage:
Change the weights using the sliders
Click on the result window to reset sliders
Hit ESC to terminate program.''')

cv2.waitKey(0)
cv2.destroyAllWindows()


# back


In [ ]:
 def efficientdet_compute_pca(self,bbox_list,confidences,im):
        #
        # * self.notebook 记录发现新的图片的W
        # * 保存数据的格式 self.save={'id':{'center':w,'LIB':[w1..wn]}}
        #
        color=[]
        identities=[]
        dths=[]
        self.frame+=1
        #维护LIB
        self.manage_lib(self.frame)
        if bbox_list!=[]:
            for bbox,c in zip(bbox_list,confidences):
                list_id,list_feature,list_distance,list_n=[],[],[],[]
                
                (x, y, w, h) = bbox
                (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
                (x,y,w,h)=(x1+x2)*0.5,(y1+y2)*0.5,abs(x1-x2),abs(y1-y2)
                im_=im[y1:y2,x1:x2]
                #im_ = self.fill_im(im_)
                im_ = cv2.resize(im_,(64,128))
                featureVector = self.compute_feature(im_)
                distance=0
            #所有图片与LIB对比 判断是否是新的id 跟新center
                for k,v in list(self.save.items()):
                    if k=='000':
                        continue
                    if v['passage']==True:
                        n=v['center']
                        distance = self.compute_cos(n,featureVector)
                        if distance <self.dth:
                            list_distance.append(distance)
                            list_id.append(k)
                            list_feature.append(featureVector)
                            list_n.append(n)
                #判断有匹配的则进行更新
                if len(list_distance)!=0:
                    min_number=list_distance.index(min(list_distance))
                    min_distance = list_distance[min_number]
                    min_distance_id = list_id[min_number]
                    min_distance_feature=list_feature[min_number]
                    min_n = list_n[min_number]
                    #dths.append(1-min(list_distance))
                    dths.append(c)
                     #检查除id_me外是否与数据库里的bbox重叠
                    ret_bbox =self.check_bboxs(bbox_list,bbox)
                    #检查重新回来的id的frame
                    #ret_id_frame = self.check_frame_id(min_distance_id,min_distance_feature,self.frame)

                    if ret_bbox==True :#and ret_id_frame==True:
                        center = (min_distance_feature*0.5)+(min_n*0.5)
                        self.save[min_distance_id]['center'] =center
                        self.save[min_distance_id]['bbox'] =bbox
                        self.save[min_distance_id]['frame'] =self.frame
                        color_ = self.check_lib_add(min_distance_feature,min_distance_id)
                        color.append(color_)
                    identities.append(min_distance_id)

                #没有匹配的图片出现
                else:
                    #检查与本帧的bbox是否重叠 和边缘是否重叠
                    ret_bbox =self.check_bboxs(bbox_list,bbox)
                    #ret_id =self.check_id(bbox_list,bbox,'#')
                    ret_lib = self.check_lib(featureVector)
                    if ret_bbox==True and ret_lib==' ':
                            id_ = self.add_id(featureVector,bbox,self.frame)
                            identities.append(id_)
                            #dths.append(1-distance)
                            dths.append(c)
                            color.append(0)
                    else:
                        color.append(0)
                        dths.append(c)
                        identities.append(' ')  
            identities = self.kalman_reid(identities,bbox_list)
        self.frame+=1
        
        return identities,dths,color

In [ ]:
 def efficientdet_compute_pca(self,bbox_list,confidences,im):
        #
        # * self.notebook 记录发现新的图片的W
        # * 保存数据的格式 self.save={'id':{'center':w,'LIB':[w1..wn]}}
        #
        color=[]
        identities=[]
        dths=[]
        #维护LIB
        self.manage_lib(self.frame)
        list_id,list_feature,list_distance,list_n=[],[],[],[]
        if bbox_list!=[]:
            for bbox,c in zip(bbox_list,confidences):
                (x, y, w, h) = bbox
                (x1, y1, x2, y2)=int(x-w/2), int(y-h/2), int(x+w/2), int(y+h/2)
                (x,y,w,h)=(x1+x2)*0.5,(y1+y2)*0.5,abs(x1-x2),abs(y1-y2)
                im_=im[y1:y2,x1:x2]
                #im_ = self.fill_im(im_)
                im_ = cv2.resize(im_,(64,128))
                #第一帧的所有创建自己的组
                #if self.frame==0:
                #    featureVector = self.compute_feature(im_)
                #    id_ = self.add_id(featureVector,bbox,self.frame)
                #    identities.append(id_)
                #    continue
                #else: 
                featureVector = self.compute_feature(im_)
                distance=0
            #所有图片与LIB对比 判断是否是新的id 分别存储到不同的list
                list_distance_buffer,list_id_buffer,list_feature_buffer,list_n_buffer=[],[],[],[]
                for k,v in list(self.save.items()):
                    if k=='000':
                        continue
                    n=v['center']
                    if v['passage']==True:
                        distance = self.compute_cos(n,featureVector)
                        list_distance_buffer.append(distance)
                        list_id_buffer.append(k)
                        list_feature_buffer.append(featureVector)
                        list_n_buffer.append(n)
                if len(list_distance_buffer)>0 and min(list_distance_buffer)<self.dth:
                    list_distance.append(list_distance_buffer)
                    list_id+=list_id_buffer
                    list_feature+=list_feature_buffer
                    list_n+=list_n_buffer
                else:
                    #distance_ = min(list_distance_buffer)
                    #检查与本帧的bbox是否重叠 和边缘是否重叠
                    ret_bbox =self.check_bboxs(bbox_list,bbox)
                    #ret_id =self.check_id(bbox_list,bbox,'#')
                    ret_lib = self.check_lib(featureVector)
                    if ret_bbox==True:
                        if ret_lib==' ':
                            id_ = self.add_id(featureVector,bbox,self.frame)
                            identities.append(id_)
                            #dths.append(1-distance_)
                            dths.append(c)
                            color.append(0)
                        else:
                            color.append(0)
                            dths.append(c)
                            #self.paassag_on(featureVector,bbox,self.frame,ret_lib)
                            identities.append(' ')
                    else:
                        color.append(0)
                        dths.append(c)
                        identities.append(' ')  

            if len(list_distance)>0 and len(bbox_list)>0:
                list_center,list_min_n,list_identities=[],[],[]
                np_distance=np.asarray(list_distance)
                #print(np_distance)
                #print(self.save.keys())
                list_Hungary = self.computer_Hungary(np_distance)
                #print(list_Hungary)
                for number in list_Hungary:
                    #number = number*0.00001
                    min_distance = np_distance.flatten().tolist().index(number)
                    list_identities.append(list_id[min_distance])
                    list_center.append(list_feature[min_distance])
                    list_min_n.append(list_n[min_distance])

                #检查重新回来的id的frame
                #ret_id_frame = self.check_frame_id(min_distance_id,min_distance_feature,self.frame)
                for bbox,conter,min_n,id_,distance_ in zip(bbox_list,list_center,list_min_n,list_identities,list_Hungary):
                     #检查除id_me外是否与数据库里的bbox重叠
                    #if distance_<self.dth:
                        ret_bbox =self.check_bboxs(bbox_list,bbox)
                        if ret_bbox==True :#and ret_id_frame==True:
                            center_ = (conter*0.5)+(min_n*0.5)
                            self.save[id_]['center'] =center_
                            self.save[id_]['bbox'] =bbox
                            self.save[id_]['frame'] =self.frame
                            color_ = self.check_lib_add(conter,id_,bbox)
                            color.append(color_)
                            dths.append(1-distance_)
                            identities.append(id_)
                        else:
                            #self.save[id_]['bbox'] =bbox
                            self.save[id_]['frame'] =self.frame
                            color_ = self.check_lib_add(conter,id_,bbox)
                            color.append(color_)
                            dths.append(1-distance_)
                            identities.append(' ')
                    
        self.frame+=1
        #保存id
        self.old_id=identities
        return identities,dths,color